In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [4]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_syllable_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffeled_syllable_train_inputs.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffeled_syllable_test_inputs.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffeled_syllable_validate_inputs.h5', other_features=True)

# syllable_letters_translator = create_syllable_letters_translator(max_syllable, syllable_dictionary, dictionary, vowels)

data = Data('sl')
data.generate_data('shuffled_syllable_reversed_train',
                   'shuffled_syllable_reversed_test',
                   'shuffled_syllable_reversed_validate')

READING CONTENT...
CONTENT READ SUCCESSFULLY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [9]:
%run ../../../prepare_data.py
gen = data.generator('train', 16)
test = next(gen)

In [10]:
# prints test word from generator
# print(X_train[0])
# print(syllable_dictionary[test[0][0][0][0].tolist().index(1)])
print(np.array(test[0][0]).shape)
for el in test[0][0][0]:
    syllable = ''
    for pos in range(0, len(el), 36):
        syllable += data.decode_x([el[pos:pos+36]], dictionary)
    print(syllable)
#     print(el)
    
#     print(syllable_dictionary[el.tolist().index(1)])

(16, 10, 252)
ša
lnej
a
ktu
ja
na






In [5]:
num_examples = len(data.x_train) # training set size

nn_output_dim = 10
nn_hdim = 516
batch_size = 16
actual_epoch = 20
num_fake_epoch = 20



conv_input_shape=(10, 252)
othr_input = (140, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
# x_conv = Conv1D(43, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(200, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])

In [6]:
history = model.fit_generator(data.generator('train', batch_size), 
                              data.x_train.shape[0]/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              validation_data=data.generator('test', batch_size), 
                              validation_steps=data.x_test.shape[0]/(batch_size * num_fake_epoch))

Epoch 1/400
1345/1344 [==============================] - 35s - loss: 0.2016 - actual_accuracy: 0.4422 - val_loss: 0.1335 - val_actual_accuracy: 0.6645
Epoch 2/400
1345/1344 [==============================] - 36s - loss: 0.1297 - actual_accuracy: 0.6335 - val_loss: 0.1065 - val_actual_accuracy: 0.7281
Epoch 3/400
1345/1344 [==============================] - 38s - loss: 0.1107 - actual_accuracy: 0.6965 - val_loss: 0.0988 - val_actual_accuracy: 0.7523
Epoch 4/400
1345/1344 [==============================] - 37s - loss: 0.0990 - actual_accuracy: 0.7373 - val_loss: 0.0884 - val_actual_accuracy: 0.7975
Epoch 5/400
1345/1344 [==============================] - 36s - loss: 0.0921 - actual_accuracy: 0.7629 - val_loss: 0.0912 - val_actual_accuracy: 0.7937
Epoch 6/400
1345/1344 [==============================] - 40s - loss: 0.0844 - actual_accuracy: 0.7847 - val_loss: 0.0833 - val_actual_accuracy: 0.8044
Epoch 7/400
1345/1344 [==============================] - 38s - loss: 0.0799 - actual_accuracy:

1345/1344 [==============================] - 44s - loss: 0.0201 - actual_accuracy: 0.9552 - val_loss: 0.1037 - val_actual_accuracy: 0.8781
Epoch 55/400
1345/1344 [==============================] - 46s - loss: 0.0192 - actual_accuracy: 0.9547 - val_loss: 0.0908 - val_actual_accuracy: 0.8831
Epoch 56/400
1345/1344 [==============================] - 48s - loss: 0.0193 - actual_accuracy: 0.9544 - val_loss: 0.1042 - val_actual_accuracy: 0.8715
Epoch 57/400
1345/1344 [==============================] - 44s - loss: 0.0178 - actual_accuracy: 0.9588 - val_loss: 0.1032 - val_actual_accuracy: 0.8758
Epoch 58/400
1345/1344 [==============================] - 48s - loss: 0.0168 - actual_accuracy: 0.9619 - val_loss: 0.1252 - val_actual_accuracy: 0.8647
Epoch 59/400
1345/1344 [==============================] - 50s - loss: 0.0175 - actual_accuracy: 0.9580 - val_loss: 0.1135 - val_actual_accuracy: 0.8708
Epoch 60/400
1345/1344 [==============================] - 45s - loss: 0.0182 - actual_accuracy: 0.959

1345/1344 [==============================] - 43s - loss: 0.0110 - actual_accuracy: 0.9759 - val_loss: 0.1181 - val_actual_accuracy: 0.8746
Epoch 108/400
1345/1344 [==============================] - 48s - loss: 0.0103 - actual_accuracy: 0.9769 - val_loss: 0.1193 - val_actual_accuracy: 0.8777
Epoch 109/400
1345/1344 [==============================] - 47s - loss: 0.0102 - actual_accuracy: 0.9772 - val_loss: 0.1155 - val_actual_accuracy: 0.8934
Epoch 110/400
1345/1344 [==============================] - 50s - loss: 0.0110 - actual_accuracy: 0.9772 - val_loss: 0.1099 - val_actual_accuracy: 0.8823
Epoch 111/400
1345/1344 [==============================] - 45s - loss: 0.0102 - actual_accuracy: 0.9778 - val_loss: 0.1256 - val_actual_accuracy: 0.8742
Epoch 112/400
1345/1344 [==============================] - 49s - loss: 0.0107 - actual_accuracy: 0.9755 - val_loss: 0.1349 - val_actual_accuracy: 0.8800
Epoch 113/400
1345/1344 [==============================] - 46s - loss: 0.0091 - actual_accuracy:

1345/1344 [==============================] - 51s - loss: 0.0076 - actual_accuracy: 0.9831 - val_loss: 0.1237 - val_actual_accuracy: 0.8938
Epoch 161/400
1345/1344 [==============================] - 54s - loss: 0.0073 - actual_accuracy: 0.9842 - val_loss: 0.1341 - val_actual_accuracy: 0.8884
Epoch 162/400
1345/1344 [==============================] - 52s - loss: 0.0077 - actual_accuracy: 0.9838 - val_loss: 0.1515 - val_actual_accuracy: 0.8762
Epoch 163/400
1345/1344 [==============================] - 54s - loss: 0.0074 - actual_accuracy: 0.9845 - val_loss: 0.1205 - val_actual_accuracy: 0.8788
Epoch 164/400
1345/1344 [==============================] - 57s - loss: 0.0076 - actual_accuracy: 0.9834 - val_loss: 0.1306 - val_actual_accuracy: 0.8838
Epoch 165/400
1345/1344 [==============================] - 49s - loss: 0.0071 - actual_accuracy: 0.9842 - val_loss: 0.1326 - val_actual_accuracy: 0.8808
Epoch 166/400
1345/1344 [==============================] - 54s - loss: 0.0069 - actual_accuracy:

1345/1344 [==============================] - 50s - loss: 0.0057 - actual_accuracy: 0.9871 - val_loss: 0.1480 - val_actual_accuracy: 0.8857
Epoch 214/400
1345/1344 [==============================] - 50s - loss: 0.0069 - actual_accuracy: 0.9858 - val_loss: 0.1288 - val_actual_accuracy: 0.8900
Epoch 215/400
1345/1344 [==============================] - 53s - loss: 0.0059 - actual_accuracy: 0.9874 - val_loss: 0.1209 - val_actual_accuracy: 0.8930
Epoch 216/400
1345/1344 [==============================] - 54s - loss: 0.0066 - actual_accuracy: 0.9857 - val_loss: 0.1299 - val_actual_accuracy: 0.8957
Epoch 217/400
1345/1344 [==============================] - 50s - loss: 0.0055 - actual_accuracy: 0.9880 - val_loss: 0.1217 - val_actual_accuracy: 0.8949
Epoch 218/400
1345/1344 [==============================] - 50s - loss: 0.0055 - actual_accuracy: 0.9875 - val_loss: 0.1133 - val_actual_accuracy: 0.8969
Epoch 219/400
1345/1344 [==============================] - 53s - loss: 0.0059 - actual_accuracy:

1345/1344 [==============================] - 55s - loss: 0.0049 - actual_accuracy: 0.9904 - val_loss: 0.1331 - val_actual_accuracy: 0.8995
Epoch 267/400
1345/1344 [==============================] - 49s - loss: 0.0055 - actual_accuracy: 0.9886 - val_loss: 0.1249 - val_actual_accuracy: 0.8934
Epoch 268/400
1345/1344 [==============================] - 53s - loss: 0.0044 - actual_accuracy: 0.9912 - val_loss: 0.1451 - val_actual_accuracy: 0.8900
Epoch 269/400
1345/1344 [==============================] - 50s - loss: 0.0048 - actual_accuracy: 0.9903 - val_loss: 0.1292 - val_actual_accuracy: 0.8984
Epoch 270/400
1345/1344 [==============================] - 51s - loss: 0.0049 - actual_accuracy: 0.9903 - val_loss: 0.1482 - val_actual_accuracy: 0.8874
Epoch 271/400
1345/1344 [==============================] - 52s - loss: 0.0051 - actual_accuracy: 0.9895 - val_loss: 0.1421 - val_actual_accuracy: 0.8861
Epoch 272/400
1345/1344 [==============================] - 50s - loss: 0.0048 - actual_accuracy:

1345/1344 [==============================] - 53s - loss: 0.0043 - actual_accuracy: 0.9901 - val_loss: 0.1342 - val_actual_accuracy: 0.8903
Epoch 320/400
1345/1344 [==============================] - 52s - loss: 0.0048 - actual_accuracy: 0.9904 - val_loss: 0.1206 - val_actual_accuracy: 0.9015
Epoch 321/400
1345/1344 [==============================] - 50s - loss: 0.0051 - actual_accuracy: 0.9889 - val_loss: 0.1326 - val_actual_accuracy: 0.890051 - actual_accu
Epoch 322/400
1345/1344 [==============================] - 57s - loss: 0.0036 - actual_accuracy: 0.9922 - val_loss: 0.1574 - val_actual_accuracy: 0.8854
Epoch 323/400
1345/1344 [==============================] - 54s - loss: 0.0039 - actual_accuracy: 0.9912 - val_loss: 0.1425 - val_actual_accuracy: 0.8907
Epoch 324/400
1345/1344 [==============================] - 57s - loss: 0.0045 - actual_accuracy: 0.9911 - val_loss: 0.1258 - val_actual_accuracy: 0.9041
Epoch 325/400
1345/1344 [==============================] - 51s - loss: 0.0041 - 

1345/1344 [==============================] - 50s - loss: 0.0034 - actual_accuracy: 0.9927 - val_loss: 0.1406 - val_actual_accuracy: 0.8934
Epoch 373/400
1345/1344 [==============================] - 49s - loss: 0.0038 - actual_accuracy: 0.9914 - val_loss: 0.1439 - val_actual_accuracy: 0.8888
Epoch 374/400
1345/1344 [==============================] - 54s - loss: 0.0043 - actual_accuracy: 0.9910 - val_loss: 0.1318 - val_actual_accuracy: 0.8880
Epoch 375/400
1345/1344 [==============================] - 53s - loss: 0.0042 - actual_accuracy: 0.9912 - val_loss: 0.1371 - val_actual_accuracy: 0.8915
Epoch 376/400
1345/1344 [==============================] - 53s - loss: 0.0035 - actual_accuracy: 0.9922 - val_loss: 0.1607 - val_actual_accuracy: 0.8888
Epoch 377/400
1345/1344 [==============================] - 53s - loss: 0.0033 - actual_accuracy: 0.9931 - val_loss: 0.1492 - val_actual_accuracy: 0.8815
Epoch 378/400
1345/1344 [==============================] - 54s - loss: 0.0044 - actual_accuracy:

In [27]:
name = '20_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [7]:
# dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels = create_dict()
# train_content, test_content, validate_content = split_content(content, 0.2, '../../internal_representations/inputs/content_shuffle_vector.h5')
# feature_dictionary = create_feature_dictionary()
# syllable_dictionary = create_syllables_dictionary(content, vowels)
# max_syllable = get_max_syllable(syllable_dictionary)


content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()
syllable_dictionary = data._create_syllables_dictionary(content, vowels)

In [8]:
%run ../../../prepare_data.py
# generate_X_and_y(dictionary, max_word, max_num_vowels, content, vowels, accetuated_vowels, feature_dictionary)